In [0]:
import os
import numpy as np
from keras.preprocessing.image import ImageDataGenerator

In [6]:
from google.colab import auth
auth.authenticate_user()

from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [7]:
cd /content/gdrive/My\ Drive/Colab\ Notebooks/visual-intelligence-course/notes/day009/

/content/gdrive/My Drive/Colab Notebooks/visual-intelligence-course/notes/day008


In [8]:
import os
import numpy as np
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255.,
                                   shear_range = 0.2, # 돌리는 정도
                                   zoom_range = 0.2,  # Zoom
                                   horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale=1./255.)

training_set = train_datagen.flow_from_directory('dataset/training_set',
                                                 target_size=(224, 224),
                                                 batch_size=20,
                                                 class_mode='binary')
test_set = test_datagen.flow_from_directory('dataset/test_set',
                                            target_size=(224,224),
                                            batch_size=20,
                                            class_mode='binary')

Found 40 images belonging to 2 classes.
Found 10 images belonging to 2 classes.


In [9]:
from keras.layers import Input, Dense, GlobalAveragePooling2D, Flatten
from keras.models import Model

from keras.applications.vgg16 import VGG16
from keras import layers
from keras import models

VGGnet=VGG16()

W0711 13:44:52.197366 139757206718336 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0711 13:44:52.224443 139757206718336 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0711 13:44:52.228975 139757206718336 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0711 13:44:52.261277 139757206718336 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:3976: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.

W0711 13:44:54.160355 139757206718336 deprecation_wrapp

In [0]:
VGGnet.summary()

In [10]:
for layer in VGGnet.layers:
  layer.trainable=False #피쳐 추출부분은 영향을 안주겠다는 의미
  
vgg_maxpool5=VGGnet.get_layer('block5_pool').output

Feature_Flatten=Flatten()(vgg_maxpool5)
dense=Dense(10,name='dense',activation='relu')(Feature_Flatten)
predictions=Dense(1,activation='sigmoid')(dense)
New_VGGmodel=Model(inputs=VGGnet.input, outputs=predictions)
New_VGGmodel.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics=['accuracy'])

New_VGGmodel.summary()

W0711 13:45:00.198385 139757206718336 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0711 13:45:00.228308 139757206718336 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/nn_impl.py:180: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [12]:
history=New_VGGmodel.fit_generator(
      training_set, #이미지 제너레이터
      steps_per_epoch=10,
      epochs=30,)
#       validation_data=validation_generator, 지금은 안만들어 놓아서 스킵
#       validation_steps=50

Epoch 1/30
10/10 [==============================] - 6s 611ms/step - loss: 1.1904 - acc: 0.6050
Epoch 2/30
10/10 [==============================] - 3s 275ms/step - loss: 0.4796 - acc: 0.7700
Epoch 3/30
10/10 [==============================] - 4s 379ms/step - loss: 0.2649 - acc: 0.9300
Epoch 4/30
10/10 [==============================] - 4s 381ms/step - loss: 0.1793 - acc: 0.9750
Epoch 5/30
10/10 [==============================] - 4s 383ms/step - loss: 0.1324 - acc: 0.9950
Epoch 6/30
10/10 [==============================] - 4s 383ms/step - loss: 0.0692 - acc: 1.0000
Epoch 7/30
10/10 [==============================] - 4s 378ms/step - loss: 0.0475 - acc: 1.0000
Epoch 8/30
10/10 [==============================] - 4s 378ms/step - loss: 0.0366 - acc: 1.0000
Epoch 9/30
10/10 [==============================] - 4s 383ms/step - loss: 0.0698 - acc: 0.9900
Epoch 10/30
10/10 [==============================] - 4s 380ms/step - loss: 0.0140 - acc: 1.0000
Epoch 11/30
10/10 [==============================

In [0]:
test_datagen=ImageDataGenerator(rescale=1./255)

test_set=test_datagen.flow_from_directory(test_dir,
                                         target_size=(224,224),
                                         batch_size=32,
                                         class_mode='binary')

Found 10 images belonging to 2 classes.


In [0]:
print(test_set[0])

(array([[[[0.10196079, 0.1764706 , 0.3019608 ],
         [0.1137255 , 0.20392159, 0.32941177],
         [0.10588236, 0.227451  , 0.34901962],
         ...,
         [0.16862746, 0.3019608 , 0.4901961 ],
         [0.14901961, 0.28235295, 0.47058827],
         [0.14509805, 0.2784314 , 0.4666667 ]],

        [[0.08235294, 0.15686275, 0.28235295],
         [0.09019608, 0.18039216, 0.30588236],
         [0.07843138, 0.19215688, 0.31764707],
         ...,
         [0.16470589, 0.29803923, 0.48627454],
         [0.15294118, 0.28627452, 0.47450984],
         [0.14901961, 0.28235295, 0.47058827]],

        [[0.1254902 , 0.18823531, 0.32156864],
         [0.12156864, 0.20000002, 0.32941177],
         [0.10196079, 0.19607845, 0.32941177],
         ...,
         [0.15686275, 0.2901961 , 0.4784314 ],
         [0.16078432, 0.29411766, 0.48235297],
         [0.16078432, 0.29411766, 0.48235297]],

        ...,

        [[0.25882354, 0.3372549 , 0.4666667 ],
         [0.2627451 , 0.34117648, 0.47058827

In [0]:
import matplotlib.pyplot as plt

output=New_VGGmodel.predict_generator(test_set,steps=1) #0이면 dog, 1이면 cat. steps도 배치사이즈라 생각해보면 됨
print(test_set.class_indices) #thretholding을 통해 값을 나누어주긴 해야함

for i in range(len(output)):
#   plt.plot(test_set[0][i]) #사진 출력해보자
  print("%.2f"%output[i])

{'cats': 0, 'dogs': 1}
1.00
0.74
0.86
0.36
1.00
1.00
1.00
1.00
1.00
0.36


In [0]:
from keras.applications.inception_v3 import InceptionV3

Incep=InceptionV3(weights='imagenet',include_top=False, input_shape=(224,224,3))

for layer in Incep.layers:
  layer.trainable=False

W0711 12:04:41.376148 140179287037824 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:1834: The name tf.nn.fused_batch_norm is deprecated. Please use tf.compat.v1.nn.fused_batch_norm instead.

W0711 12:04:42.475665 140179287037824 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:3980: The name tf.nn.avg_pool is deprecated. Please use tf.nn.avg_pool2d instead.



87916544/87910968 [==============================] - 4s 0us/step


In [0]:
incep_maxpool4=Incep.layers[-2].output

Feature_Flatten=Flatten()(incep_maxpool4)
dense=Dense(10,name='dense',activation='relu')(Feature_Flatten)
predictions=Dense(1,activation='sigmoid')(dense)
New_incep=Model(inputs=Incep.input, outputs=predictions)
New_incep.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics=['accuracy'])

New_incep.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 111, 111, 32) 864         input_2[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 111, 111, 32) 96          conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 111, 111, 32) 0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
conv2d_2 (